### Assignment 15 - Random Forest

#### 1st Problem Statement
A cloth manufacturing company is interested to know about the segment or attributes causes high sale. 

Approach - A decision tree can be built with target variable Sale (we will first convert it in categorical variable) & all other variable will be independent in the analysis.  

Dataset --> Company_Data.csv

In [1]:
#Importing the Required Libraries
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [2]:
#Loading The Dataset
Dataset = pd.read_csv("C:/Users/Akaash/Downloads/Company_Data.csv")
Dataset.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [3]:
#Checking For NA Values & Dtypes of Columns
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


Inference: There are No NA Values Present in the Dataset, 

"Sales" is Our Y Variable as Per problem statement, Which is a Continuous variable, So Converting it by Creating a buckets of levels i.e.(Median of Sales)

There are 3 X Variable of object Dtypes so will use dummies to Convert them to numerical types

In [4]:
#Getting the Median of Sales to make Buckets of Data
print(Dataset['Sales'].median())

7.49


In [5]:
#Creating a new Column with two bucket of Data for Sales
Dataset["sales"]= "<=7.49"
#Setting Bucket Based on Median Value of Original Sales Variables 
Dataset.loc[Dataset["Sales"]>=7.49,"sales"]=">=7.49"
Dataset.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,sales
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes,>=7.49
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes,>=7.49
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes,>=7.49
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes,<=7.49
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No,<=7.49


Inference: Now the New 'sales' column has Two Label/Bucket Assigned based on Median of Sales (Y) Variable, then will Drop the Original Sales Variable

In [6]:
#Dropping Sales column from the data 
Dataset.drop(["Sales"],axis=1,inplace = True)
Dataset.head()

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,sales
0,138,73,11,276,120,Bad,42,17,Yes,Yes,>=7.49
1,111,48,16,260,83,Good,65,10,Yes,Yes,>=7.49
2,113,35,10,269,80,Medium,59,12,Yes,Yes,>=7.49
3,117,100,4,466,97,Medium,55,14,Yes,Yes,<=7.49
4,141,64,3,340,128,Bad,38,13,Yes,No,<=7.49


Inference: Original 'Sales' is Dropped.

Now Will Do Encoding for Y Variable Label Encoding, and For X Variable One Hot Encoding

In [7]:
#Applying lable encoder to sales column
label_encoder = preprocessing.LabelEncoder()
Dataset['sales'] = label_encoder.fit_transform(Dataset['sales']) 
Dataset.head()

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,sales
0,138,73,11,276,120,Bad,42,17,Yes,Yes,1
1,111,48,16,260,83,Good,65,10,Yes,Yes,1
2,113,35,10,269,80,Medium,59,12,Yes,Yes,1
3,117,100,4,466,97,Medium,55,14,Yes,Yes,0
4,141,64,3,340,128,Bad,38,13,Yes,No,0


Inference: Label Encoding is Applied, Now will do One Hot Encoding For X Variable

In [8]:
#Get Dummies -- One Hot Encoding
Final_df = pd.get_dummies(Dataset)
Final_df.head()

,CompPrice,Income,Advertising,Population,Price,Age,Education,sales,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium,Urban_No,Urban_Yes,US_No,US_Yes
0,138,73,11,276,120,42,17,1,1,0,0,0,1,0,1
1,111,48,16,260,83,65,10,1,0,1,0,0,1,0,1
2,113,35,10,269,80,59,12,1,0,0,1,0,1,0,1
3,117,100,4,466,97,55,14,0,0,0,1,0,1,0,1
4,141,64,3,340,128,38,13,0,1,0,0,0,1,1,0


In [9]:
# seprating x and y variables 
x= pd.concat([Final_df.iloc[:,0:7],Final_df.iloc[:,8:]],axis = 1)
y= Final_df.iloc[:,7]

In [10]:
# to check unique values in specified column
print(Final_df['sales'].unique())
# to count total values for distinct/unique value
print(Final_df.sales.value_counts())

[1 0]
1    201
0    199
Name: sales, dtype: int64


Inference: We Can Say This is almost a Balanced Dataset.

When the Observation/Values are Equally Distributed along the all the Classes then it called Balanced Dataset.when They are not Equally Distributed then it is Called Inbalance Dataset 

Note: 

Claas 1 Refers to Sales High 

Class 0 Refers To Sales Low

#### Grid Search CV

In [11]:
#Importing Required Library
from sklearn.model_selection import GridSearchCV
#Creating Random Forest instance
clf = RandomForestClassifier()

#Defining Grid Search Parameter
param_grid = [{'n_estimators':[50,90,100],'max_features':[3,4,5]}]

#Creating Grid Search instance and fitting the Model
gsv = GridSearchCV(clf,param_grid,cv=5)
gsv.fit(x,y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid=[{'max_features': [3, 4, 5],
                          'n_estimators': [50, 90, 100]}])

In [12]:
#Getting Search Results From Grid Search
gsv.best_params_ , gsv.best_score_ 

({'max_features': 5, 'n_estimators': 100}, 0.7975)

Inference: Through Grid Search We Got values of parameters max_features = 5, n_estimators = 100 with Accuracy of 79.75%, So Now Building the Random Forest Model With These Hyperparameter Values

#### Random Forest Model

In [13]:
#Training the Model ~ Random Forest 
num_trees = 100
max_features = 5 
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features) # Hyperparameter Values Got Through GridCV

In [14]:
# Creating K-Fold Instance
kfold = KFold(n_splits=5, random_state=7,shuffle = True)

In [15]:
#Predicting & Getting the Model Score with kfold
results = cross_val_score(model, x, y, cv=kfold)
# Accuracy for Each K-Fold
results

array([0.75 , 0.8  , 0.825, 0.875, 0.775])

In [16]:
#Average of All k-fold Accuracy to get Final model Accuracy
results.mean()*100.0

80.5

Inference: Using The Random Forest the Final Accuracy is 80.5%

#### 2nd Problem Statement
Use decision trees to prepare a model on fraud data 
treating those who have taxable_income <= 30000 as "Risky" and others are "Good" 

Dataset --> Fraud_check.csv

In [17]:
#Importing the Required Libraries
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [18]:
#Loading The Dataset
Dataset = pd.read_csv("C:/Users/Akaash/Downloads/Fraud_check.csv")
Dataset.head()

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


Inference: Columns/ Variable Names are not right so, Renaming them first

In [19]:
Dataset.rename(columns={"Marital.Status":"Marital_Status","Taxable.Income":"Taxable_Income",
                        "City.Population":"City_Population","Work.Experience":"Work_Experience"},inplace=True)
Dataset.head()

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


Inference: Now Columns Name are Correct

In [20]:
#Checking For NA Value and Dtypes of Columns
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital_Status   600 non-null    object
 2   Taxable_Income   600 non-null    int64 
 3   City_Population  600 non-null    int64 
 4   Work_Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


Inference: There are No NA Values Present in the Dataset, 

"Taxable_Income" is Our Y Variable as Per problem statement, Which is a Continuous variable, So Converting it by Creating a buckets of levels i.e. <= 30000 as "Risky" and others are "Good" 

There are 3 X Variable of object Dtypes so will use dummies to Convert them to numerical types

In [21]:
##Converting the Taxable income variable to bucketing. 

#Creating a new Column with two bucket of Data for Taxable_Income
Dataset["taxable_income"]="<=30000"

#Setting Bucket Based on Median Value of Original Sales Variables 
Dataset.loc[Dataset["Taxable_Income"]>=30000,"taxable_income"]="Good"
Dataset.loc[Dataset["Taxable_Income"]<=30000,"taxable_income"]="Risky"

##Droping the Taxable income variable
Dataset.drop(["Taxable_Income"],axis=1,inplace=True)

Dataset.head()

,Undergrad,Marital_Status,City_Population,Work_Experience,Urban,taxable_income
0,NO,Single,50047,10,YES,Good
1,YES,Divorced,134075,18,YES,Good
2,NO,Married,160205,30,YES,Good
3,YES,Single,193264,15,YES,Good
4,NO,Married,27533,28,NO,Good


Inference: Now Will Do Encoding for Y Variable Label Encoding, and For X Variable One Hot Encoding

In [22]:
#Applying lable encoder to sales column
label_encoder = preprocessing.LabelEncoder()
Dataset['taxable_income'] = label_encoder.fit_transform(Dataset['taxable_income']) 
Dataset.head()

,Undergrad,Marital_Status,City_Population,Work_Experience,Urban,taxable_income
0,NO,Single,50047,10,YES,0
1,YES,Divorced,134075,18,YES,0
2,NO,Married,160205,30,YES,0
3,YES,Single,193264,15,YES,0
4,NO,Married,27533,28,NO,0


Inference: Lets assume: taxable_income <= 30000 as “Risky=0” and others are “Good=1”, Now Encoding For X Variable

In [23]:
#Get Dummies -- One Hot Encoding
Final_df = pd.get_dummies(Dataset)
Final_df.head()

,City_Population,Work_Experience,taxable_income,Undergrad_NO,Undergrad_YES,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Urban_NO,Urban_YES
0,50047,10,0,1,0,0,0,1,0,1
1,134075,18,0,0,1,1,0,0,0,1
2,160205,30,0,1,0,0,1,0,0,1
3,193264,15,0,0,1,0,0,1,0,1
4,27533,28,0,1,0,0,1,0,1,0


Inference: After Apply Encoding the Dataset is Ready for Decision tree, Lets Separate X & Y Variable Now

In [24]:
# seprating x and y variables 
x= pd.concat([Final_df.iloc[:,0:2],Final_df.iloc[:,3:]],axis = 1)
y= Final_df.iloc[:,2]

In [25]:
# to check unique values in specified column
print(Final_df['taxable_income'].unique())
# to count total values for distinct/unique value
print(Final_df.taxable_income.value_counts())

[0 1]
0    476
1    124
Name: taxable_income, dtype: int64


Inferences: X & Y Variables are Separated Now

#### Grid Search CV

In [26]:
#Importing Required Library
from sklearn.model_selection import GridSearchCV
#Creating Random Forest instance
clf = RandomForestClassifier()

#Defining Grid Search Parameter
param_grid = [{'n_estimators':[50,90,100],'max_features':[3,4,5]}]

#Creating Grid Search instance and fitting the Model
gsv = GridSearchCV(clf,param_grid,cv=5)
gsv.fit(x,y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid=[{'max_features': [3, 4, 5],
                          'n_estimators': [50, 90, 100]}])

In [27]:
#Getting Search Results From Grid Search
gsv.best_params_ , gsv.best_score_ 

({'max_features': 5, 'n_estimators': 50}, 0.7449999999999999)

Inference: Through Grid Search We Got values of parameters max_features = 5, n_estimators = 50 with Accuracy of 74.49%, So Now Building the Random Forest Model With These Hyperparameter Values

#### Random Forest Model

In [28]:
#Training the Model ~ Random Forest 
num_trees = 50
max_features = 5
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features) # Hyperparameter Values Got Through GridCV

In [29]:
# Creating K-Fold Instance
kfold = KFold(n_splits=5, random_state=7,shuffle = True)

In [30]:
#Predicting & Getting the Model Score with kfold
results = cross_val_score(model, x, y, cv=kfold)
# Accuracy for Each K-Fold
results

array([0.71666667, 0.83333333, 0.725     , 0.7       , 0.79166667])

In [31]:
#Average of All k-fold Accuracy to get Final model Accuracy
results.mean()*100.0

75.33333333333331

Inference: Using The Random Forest the Final Accuracy is 75.33%